## Lesson 5 - Model
 - 이번 실습 자료에서는 강의시간에 다루었던 파이토치 모델을 정의하는 방법에 대해 실습하겠습니다.
 - 파이토치 모델은 기본적으로 `nn.Module` 클래스를 상속하여 사용합니다.
     - [공식문서](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)에 따르면 `nn.Module` 은 다음과 같은 기능을 합니다
     ```
     Base class for all neural network modules.
     Your models should also subclass this class.
     Modules can also contain other Modules, allowing to nest them in a tree structure. You can assign the submodules as regular attributes:
     ```

In [1]:
from pprint import pprint

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3, bias=True)
        self.bn1 = nn.BatchNorm2d(num_features=3)
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=3, bias=False)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        return F.relu(self.conv2(x))

In [3]:
model = Model()
model

Model(
  (conv1): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1), bias=False)
)

### 모델 디버깅
 - 파이토치 모델들은 다음과 같읕 방법들을 통해 파라미터를 눈으로 확인할 수 있습니다.

In [4]:
# 1. named_parameters() 를 이용하는 방식
for param, weight in model.named_parameters():
    print(f"{param:20} - size: {weight.size()}")
    print(weight)
    print("-" * 100)
    print()

conv1.weight         - size: torch.Size([3, 1, 3, 3])
Parameter containing:
tensor([[[[-0.2500, -0.1243, -0.2524],
          [-0.0348,  0.2156, -0.2962],
          [-0.0547,  0.0366, -0.3217]]],


        [[[ 0.2274,  0.1173,  0.2428],
          [ 0.3236, -0.0628, -0.0311],
          [ 0.1467, -0.1174,  0.2254]]],


        [[[ 0.2126,  0.2160, -0.0055],
          [ 0.0955, -0.1705,  0.2118],
          [ 0.1571, -0.1833,  0.2524]]]], requires_grad=True)
----------------------------------------------------------------------------------------------------

conv1.bias           - size: torch.Size([3])
Parameter containing:
tensor([-0.3020, -0.0305,  0.3019], requires_grad=True)
----------------------------------------------------------------------------------------------------

bn1.weight           - size: torch.Size([3])
Parameter containing:
tensor([1., 1., 1.], requires_grad=True)
----------------------------------------------------------------------------------------------------

bn1.b

In [5]:
# 2. 멤버 변수를 이용하여 직접 access 하는 방법
print(model.conv1.weight)
print(model.conv1.bias)

Parameter containing:
tensor([[[[-0.2500, -0.1243, -0.2524],
          [-0.0348,  0.2156, -0.2962],
          [-0.0547,  0.0366, -0.3217]]],


        [[[ 0.2274,  0.1173,  0.2428],
          [ 0.3236, -0.0628, -0.0311],
          [ 0.1467, -0.1174,  0.2254]]],


        [[[ 0.2126,  0.2160, -0.0055],
          [ 0.0955, -0.1705,  0.2118],
          [ 0.1571, -0.1833,  0.2524]]]], requires_grad=True)
Parameter containing:
tensor([-0.3020, -0.0305,  0.3019], requires_grad=True)


### 학습된 모델 저장하기
 - `torch.save(model.state_dict(), save_path)`

In [6]:
import os

save_folder = "./runs/"
save_path = os.path.join(save_folder, "best.pth")   # ./runs/best.pth
os.makedirs(save_folder, exist_ok=True)  

torch.save(model.state_dict(), save_path)
print(f"{save_path} 폴더에 모델이 성공적으로 저장되었습니다.")
print(f"해당 폴더의 파일 리스트: {os.listdir(save_folder)}")

./runs/best.pth 폴더에 모델이 성공적으로 저장되었습니다.
해당 폴더의 파일 리스트: ['best.pth']


### 저장된 모델 불러오기
 - model.load_state_dict(torch.load(save_path))

In [7]:
new_model = Model()
new_model.load_state_dict(torch.load(save_path))
print(f"{save_path} 에서 성공적으로 모델을 load 하였습니다.")

./runs/best.pth 에서 성공적으로 모델을 load 하였습니다.


#### 저장된 모델이 잘 불러와졌는지 확인해봅시다

In [8]:
for (name, trained_weight), (_, saved_weight) in zip(model.named_parameters(), new_model.named_parameters()):
    is_equal = torch.equal(trained_weight, saved_weight)
    print(f"파라미터 {name:15} 에 대하여 trained 모델과 load 된 모델의 값이 같나요? -> {is_equal}")

파라미터 conv1.weight    에 대하여 trained 모델과 load 된 모델의 값이 같나요? -> True
파라미터 conv1.bias      에 대하여 trained 모델과 load 된 모델의 값이 같나요? -> True
파라미터 bn1.weight      에 대하여 trained 모델과 load 된 모델의 값이 같나요? -> True
파라미터 bn1.bias        에 대하여 trained 모델과 load 된 모델의 값이 같나요? -> True
파라미터 conv2.weight    에 대하여 trained 모델과 load 된 모델의 값이 같나요? -> True


#### state_dict() 이 무엇인가요?
 - 모델의 저장과 로딩에 `state_dict()` 을 사용하는데, 기능이 무엇인가요?
 - 기본적으로 위에서 살펴본 `.named_parameters()` 와 매우 유사합니다
 - model parameter 를 Key 로 가지고, model weights 를 Value 로 가지는 파이썬 딕셔너리일 뿐입니다. 
   (정확한 Type 은 파이썬 내장 라이브러리 collections.OrderDict 입니다)

In [9]:
for param, weight in model.state_dict().items():
    print(f"파라미터 네임 {param:25} / 사이즈: {weight.size()}")
    print(weight)
    print("-" * 100, end="\n\n")

파라미터 네임 conv1.weight              / 사이즈: torch.Size([3, 1, 3, 3])
tensor([[[[-0.2500, -0.1243, -0.2524],
          [-0.0348,  0.2156, -0.2962],
          [-0.0547,  0.0366, -0.3217]]],


        [[[ 0.2274,  0.1173,  0.2428],
          [ 0.3236, -0.0628, -0.0311],
          [ 0.1467, -0.1174,  0.2254]]],


        [[[ 0.2126,  0.2160, -0.0055],
          [ 0.0955, -0.1705,  0.2118],
          [ 0.1571, -0.1833,  0.2524]]]])
----------------------------------------------------------------------------------------------------

파라미터 네임 conv1.bias                / 사이즈: torch.Size([3])
tensor([-0.3020, -0.0305,  0.3019])
----------------------------------------------------------------------------------------------------

파라미터 네임 bn1.weight                / 사이즈: torch.Size([3])
tensor([1., 1., 1.])
----------------------------------------------------------------------------------------------------

파라미터 네임 bn1.bias                  / 사이즈: torch.Size([3])
tensor([0., 0., 0.])
-----------------

In [10]:
from collections import OrderedDict
print(f"model.state_dict() 의 Type : {type(model.state_dict())}")
isinstance(model.state_dict(), OrderedDict)

model.state_dict() 의 Type : <class 'collections.OrderedDict'>


True

#### `named_parameters()` 을 안쓰고 `state_dict()` 을 사용하는 이유가 무언인가요? (둘이 뭐가 다른가요)
 - `named_parameters()` : returns only parameters
 - `state_dict()`: returns both parameters and buffers (e.g. BN runnin_mean, running_var)
 
 [Reference](https://stackoverflow.com/a/54747245)

In [11]:
pprint([name for (name, param) in model.named_parameters()])  # named_parameters() : returns only parameters
print()
pprint(list(model.state_dict().keys()))                       # state_dict(): retuns both parameters and buffers

['conv1.weight', 'conv1.bias', 'bn1.weight', 'bn1.bias', 'conv2.weight']

['conv1.weight',
 'conv1.bias',
 'bn1.weight',
 'bn1.bias',
 'bn1.running_mean',
 'bn1.running_var',
 'bn1.num_batches_tracked',
 'conv2.weight']


### CPU vs GPU
 - Pytorch 텐서(데이터)는 다양한 프로세서(CPU, GPU, TPU) 에서 연산 및 학습이 가능합니다.
 - 따라서, 특정 프로세서에서 학습을 진행하고 싶은 경우 해당 프로세스를 명시적으로 지정해주어야 합니다.
 - 이는 해당 텐서(데이터)를 특정 프로세스의 메모리에 load 또는 해당 프로세스의 메모리로 이동하는 것을 의미합니다.
 - 따라서, 연산하는 텐서들의 디바이스가 같아야만 연산이 가능합니다. 그렇지 않을 경우 에러가 발생합니다.

#### 새로운 텐서 생성

In [12]:
data = torch.randn(2,2, device=torch.device('cpu'))     # CPU 에 새로운 텐서 생성
print(f"데이터 디바이스: {data.device}")

data = torch.randn(2,2, device=torch.device('cuda:0'))  # GPU 0번에 새로운 텐서 생성
print(f"데이터 디바이스: {data.device}")

data = torch.randn(2,2)                                 # device 를 따로 지정하지 않으면 default 로 CPU 에 생성됩니다.
print(f"데이터 디바이스: {data.device}")

데이터 디바이스: cpu
데이터 디바이스: cuda:0
데이터 디바이스: cpu


#### 이미 생성되어 있는 텐서를 다른 프로세스의 메모리로 이동하는 것도 가능합니다
#### .cpu()
모든 모델의 파라미터와 버터를 CPU 메모리로 이동

In [13]:
model.cpu()
for weight in model.parameters():
    print(f"파라미터 디바이스: {weight.device}")

파라미터 디바이스: cpu
파라미터 디바이스: cpu
파라미터 디바이스: cpu
파라미터 디바이스: cpu
파라미터 디바이스: cpu


#### .cuda()
모든 모델의 파라미터와 버터를 GPU 메모리로 이동

In [14]:
model.cuda()
for weight in model.parameters():
    print(f"파라미터 디바이스: {weight.device}")

파라미터 디바이스: cuda:0
파라미터 디바이스: cuda:0
파라미터 디바이스: cuda:0
파라미터 디바이스: cuda:0
파라미터 디바이스: cuda:0


#### .to()
파라미터 또는 버퍼 메모리를 다음 프로세스로 이동

In [15]:
device_options = ['cpu', 'cuda']
for device_option in device_options:
    device = torch.device(device_option)
    model.to(device)
    
    print(f"파라미터 디바이스를 {device_option} 로 변경")
    for weight in model.parameters():
        print(f"파라미터 디바이스: {weight.device}")
    print()

파라미터 디바이스를 cpu 로 변경
파라미터 디바이스: cpu
파라미터 디바이스: cpu
파라미터 디바이스: cpu
파라미터 디바이스: cpu
파라미터 디바이스: cpu

파라미터 디바이스를 cuda 로 변경
파라미터 디바이스: cuda:0
파라미터 디바이스: cuda:0
파라미터 디바이스: cuda:0
파라미터 디바이스: cuda:0
파라미터 디바이스: cuda:0



#### Cautions

새로운 텐서를 GPU 에 생성하고 싶은 경우 `torch.randn(2,2).cuda()` 처럼 생성하면

1) CPU 메모리에 텐서를 생성 2) CPU -> GPU 메모리로 값을 이동하는 과정이 일어나면서 cost efficient 하지 못합니다

`torch.randn(2,2, device=torch.device('cuda:0'))` 와 같이 처음부터 GPU 메모리에 생성하는 것을 권장합니다.

#### Cautions
 - 연산하는 두 개의 텐서는 반드시 같은 device 에 존재하여야 합니다.
 - 그렇지 않으면 에러가 발생합니다.

In [16]:
data1 = torch.randn(2,2, device=torch.device('cpu'))
data2 = torch.randn(2,2, device=torch.device('cpu'))
print(data1 + data2)  # 두 텐서가 같은 device(CPU) 에 있기에 연산이 가능합니다.

tensor([[-0.1846, -0.3361],
        [ 3.0285,  1.1365]])


In [17]:
data1 = torch.randn(2,2, device=torch.device('cpu'))
data2 = torch.randn(2,2, device=torch.device('cuda'))
print(data1 + data2)  # 두 텐서가 다른 device(CPU, GPU) 에 있기에 연산이 불가능합니다.

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

### forward
 - nn.Module 을 상속한 객체를 직접 호출할 때 수행하는 연산을 정의합니다.
 - `model(input)` 을 통해 모델의 예측값을 계산할 수 있습니다.
 - Defines the computation performed at every call

In [ ]:
dummy_input = torch.randn(1, 1, 12, 12, device=device)
model.to(device)
output = model(dummy_input)
print(f"모델 output 사이즈: {output.size()}")
print(output)

#### Cautions
 - 위에서 말씀드린 것과 같은 원리로 모델과 인풋의 device 는 반드시 같아야 합니다.

In [ ]:
cpu_device = torch.device('cpu')
gpu_device = torch.device('cuda')

# device is same
dummy_input = dummy_input.to(gpu_device)
model.to(gpu_device)
output = model(dummy_input)  # 잘 작동합니다 
print(f"모델 ouput 사이즈: {output.size()}")

In [ ]:
dummy_input = dummy_input.to(cpu_device)
model.to(gpu_device)

# device is different
# RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same
output = model(dummy_input)  # 에러 발생
print(f"모델 ouput 사이즈: {output.size()}")

### requires_grad()
 - autograd 가 해당 모델의 연산을 기록할지를 결정합니다
 - false 일 시, 수행하는 연산을 기록하지 않고 따라서 역전파가 되지 않아 학습에서 제외됩니다.
 - Change if autograd should record operations on parameters in this module.

In [ ]:
# requires_grad = False
model.requires_grad_(requires_grad=False)
for param, weight in model.named_parameters():
    print(f"파라미터 {param:15} 가 gradient 를 tracking 하나요? -> {weight.requires_grad}")

In [ ]:
# requires_grad = True
model.requires_grad_(requires_grad=True)
for param, weight in model.named_parameters():
    print(f"파라미터 {param:15} 가 gradient 를 tracking 하나요? -> {weight.requires_grad}")

### train(), eval()
 - 모델을 training(evaluation) 모드로 전환합니다.
 - training 과 evaluation 이 다르게 작용하는 모듈들(Dropout, BatchNorm) 에 영향을 줍니다.
 - 학습 단계에서는 training 모드로, 인퍼런스 단계에서는 eval 모드로 전환해주어야 합니다.
 - [아래](https://github.com/pytorch/pytorch/blob/1.6/torch/nn/modules/batchnorm.py#L110-L117)는 BatchNorm2d 의 파이토치 구현입니다. `self.training=True` 일 경우에만, `running_mean`, `running_var` 을 tracking 합니다.
 
```
if self.training and self.track_running_stats:
    # TODO: if statement only here to tell the jit to skip emitting this when it is None
    if self.num_batches_tracked is not None:
        self.num_batches_tracked = self.num_batches_tracked + 1
        if self.momentum is None:  # use cumulative moving average
            exponential_average_factor = 1.0 / float(self.num_batches_tracked)
        else:  # use exponential moving average
            exponential_average_factor = self.momentum
```

In [ ]:
model.train()  # train mode 로 전환
print(f"model.bn1.training: {model.bn1.training}")

In [ ]:
model.eval()  # eval mode 로 전환
print(f"model.bn1.training: {model.bn1.training}")

### 파이토치 공식 문서에서 nn.Module 에 관한 더 많은 정보를 얻을 수 있습니다.
https://pytorch.org/docs/stable/generated/torch.nn.Module.html

궁금증이 생기면 공식 문서를 참고하는걸 강력 추천합니다.